In [1]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [2]:
import tensorflow as tf

import numpy as np
import os
import time

In [3]:
path_to_file = "data1.txt"

### Read the data

First, look in the text:

In [4]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 5869207 characters


In [5]:
import re
temp = ""

for i, each in enumerate(text):
  if each in "()[]{}.,*#'\"=-+:/":
    temp +=  f" {each} "
  else:
    temp += each
text = temp
text = re.sub('\\r\\n|\\r(?!\\n)', ' \r\n ', text)


In [6]:
# Take a look at the first 250 characters in text
print(text[:400])

monkey materials and tools 
 1 .  yarn :  yarnart jeans ,  colours :  black  ( 53 )  ,  light grey  ( 46 )  ,  dark grey  ( 28 )  ,  orange  ( 85 )  ,  white  ( 03 )  ,  yellow  ( 88 )  .  small quantity of white thin fluffy yarn  ( mohair ,  angora )  .  size of the toy made with this yarn is about 18 cm .  
 2 .  crochet hook according to your yarn  ( i used clover 1 , 75 )  .  
 3 .  yarn ne


In [7]:
# text_sep = tf.strings.split(example_texts, sep=" ")

text_sep = text.split(" ")
text_sep

['monkey',
 'materials',
 'and',
 'tools',
 '\r\n',
 '1',
 '.',
 '',
 'yarn',
 ':',
 '',
 'yarnart',
 'jeans',
 ',',
 '',
 'colours',
 ':',
 '',
 'black',
 '',
 '(',
 '53',
 ')',
 '',
 ',',
 '',
 'light',
 'grey',
 '',
 '(',
 '46',
 ')',
 '',
 ',',
 '',
 'dark',
 'grey',
 '',
 '(',
 '28',
 ')',
 '',
 ',',
 '',
 'orange',
 '',
 '(',
 '85',
 ')',
 '',
 ',',
 '',
 'white',
 '',
 '(',
 '03',
 ')',
 '',
 ',',
 '',
 'yellow',
 '',
 '(',
 '88',
 ')',
 '',
 '.',
 '',
 'small',
 'quantity',
 'of',
 'white',
 'thin',
 'fluffy',
 'yarn',
 '',
 '(',
 'mohair',
 ',',
 '',
 'angora',
 ')',
 '',
 '.',
 '',
 'size',
 'of',
 'the',
 'toy',
 'made',
 'with',
 'this',
 'yarn',
 'is',
 'about',
 '18',
 'cm',
 '.',
 '',
 '\r\n',
 '2',
 '.',
 '',
 'crochet',
 'hook',
 'according',
 'to',
 'your',
 'yarn',
 '',
 '(',
 'i',
 'used',
 'clover',
 '1',
 ',',
 '75',
 ')',
 '',
 '.',
 '',
 '\r\n',
 '3',
 '.',
 '',
 'yarn',
 'needle',
 ',',
 '',
 'sewing',
 'needle',
 '.',
 '',
 '\r\n',
 '4',
 '.',
 '',
 'brown',
 

In [8]:
# The unique characters in the file
vocab = sorted(set(text_sep))
print(f'{len(vocab)} unique characters')

6392 unique characters


In [ ]:
vocab

In [9]:
import os
from google.colab import files

file = open('vocab.txt','w')
for item in vocab:
	file.write(item+"\n")
file.close()

files.download("/content/vocab.txt")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Process the text

### Vectorize the text

Before training, you need to convert the strings to a numerical representation. 

The `tf.keras.layers.StringLookup` layer can convert each character into a numeric ID. It just needs the text to be split into tokens first.

In [10]:
# example_texts = ['ab \r\n efg', 'brf']

# chars = tf.strings.unicode_split(vocab, input_encoding='UTF-8')
# chars

strten = tf.strings.split(vocab, sep=' ')
strten


<tf.RaggedTensor [[b''],
 [b'\r\n'],
 [b'!'],
 ...,
 [b'~'],
 [b'~16cm'],
 [b'~23']]>

Now create the `tf.keras.layers.StringLookup` layer:

In [11]:
ids_from_strings = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

It converts from tokens to character IDs:

In [12]:
ids = ids_from_strings(strten)
ids

<tf.RaggedTensor [[1],
 [2],
 [3],
 ...,
 [6390],
 [6391],
 [6392]]>

Since the goal of this tutorial is to generate text, it will also be important to invert this representation and recover human-readable strings from it. For this you can use `tf.keras.layers.StringLookup(..., invert=True)`.  

Note: Here instead of passing the original vocabulary generated with `sorted(set(text))` use the `get_vocabulary()` method of the `tf.keras.layers.StringLookup` layer so that the `[UNK]` tokens is set the same way.

In [13]:
strings_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_strings.get_vocabulary(), invert=True, mask_token=None)

This layer recovers the characters from the vectors of IDs, and returns them as a `tf.RaggedTensor` of characters:

In [14]:
strings = strings_from_ids(ids)
strings

<tf.RaggedTensor [[b''],
 [b'\r\n'],
 [b'!'],
 ...,
 [b'~'],
 [b'~16cm'],
 [b'~23']]>

You can `tf.strings.reduce_join` to join the characters back into strings. 

In [15]:
tf.strings.reduce_join(strten, axis=-1, separator=' ').numpy()

array([b'', b'\r\n', b'!', ..., b'~', b'~16cm', b'~23'], dtype=object)

In [16]:
def text_from_ids(ids):
  return tf.strings.reduce_join(strings_from_ids(ids), axis=-1, separator=" ")

### The prediction task

Given a character, or a sequence of characters, what is the most probable next character? This is the task you're training the model to perform. The input to the model will be a sequence of characters, and you train the model to predict the output—the following character at each time step.

Since RNNs maintain an internal state that depends on the previously seen elements, given all the characters computed until this moment, what is the next character?


### Create training examples and targets

Next divide the text into example sequences. Each input sequence will contain `seq_length` characters from the text.

For each input sequence, the corresponding targets contain the same length of text, except shifted one character to the right.

So break the text into chunks of `seq_length+1`. For example, say `seq_length` is 4 and our text is "Hello". The input sequence would be "Hell", and the target sequence "ello".

To do this first use the `tf.data.Dataset.from_tensor_slices` function to convert the text vector into a stream of character indices.

In [17]:

all_ids = ids_from_strings(tf.strings.split(text, sep=' '))
all_ids

<tf.Tensor: shape=(2403746,), dtype=int64, numpy=array([3908, 3793, 1291, ..., 2480, 4297, 2769])>

In [18]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [19]:
for ids in ids_dataset.take(10):
    print(strings_from_ids(ids).numpy().decode('utf-8'))

monkey
materials
and
tools


1
.

yarn
:


In [20]:
seq_length = 100


The `batch` method lets you easily convert these individual characters to sequences of the desired size.

In [21]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(strings_from_ids(seq))

tf.Tensor(
[b'monkey' b'materials' b'and' b'tools' b'\r\n' b'1' b'.' b'' b'yarn' b':'
 b'' b'yarnart' b'jeans' b',' b'' b'colours' b':' b'' b'black' b'' b'('
 b'53' b')' b'' b',' b'' b'light' b'grey' b'' b'(' b'46' b')' b'' b',' b''
 b'dark' b'grey' b'' b'(' b'28' b')' b'' b',' b'' b'orange' b'' b'(' b'85'
 b')' b'' b',' b'' b'white' b'' b'(' b'03' b')' b'' b',' b'' b'yellow' b''
 b'(' b'88' b')' b'' b'.' b'' b'small' b'quantity' b'of' b'white' b'thin'
 b'fluffy' b'yarn' b'' b'(' b'mohair' b',' b'' b'angora' b')' b'' b'.' b''
 b'size' b'of' b'the' b'toy' b'made' b'with' b'this' b'yarn' b'is'
 b'about' b'18' b'cm' b'.' b'' b'\r\n' b'2'], shape=(101,), dtype=string)


It's easier to see what this is doing if you join the tokens back into strings:

In [22]:
temp = ""
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy()) 
  # for word in text_from_ids(seq):
  #   temp += word.numpy()
  # print(temp)
 

b'monkey materials and tools \r\n 1 .  yarn :  yarnart jeans ,  colours :  black  ( 53 )  ,  light grey  ( 46 )  ,  dark grey  ( 28 )  ,  orange  ( 85 )  ,  white  ( 03 )  ,  yellow  ( 88 )  .  small quantity of white thin fluffy yarn  ( mohair ,  angora )  .  size of the toy made with this yarn is about 18 cm .  \r\n 2'
b'.  crochet hook according to your yarn  ( i used clover 1 , 75 )  .  \r\n 3 .  yarn needle ,  sewing needle .  \r\n 4 .  brown thread for embroidering nose and eyebrows .  \r\n 5 .  button eyes diameter 5 mm ,  sewing thread of black colour to sew the eyes .  \r\n 6 .  poliester fiberfill .  \r\n work in continuos rounds unless otherwise instructed \r\n dauc materials and tools \r\n 1 .  yarn :  yarnart jeans ,  colours :'
b' black  ( 53 )  ,  light grey  ( 46 )  ,  dark grey  ( 28 )  ,  orange  ( 85 )  ,  white  ( 03 )  ,  yellow  ( 88 )  .  small quantity of white thin fluffy yarn  ( mohair ,  angora )  .  size of the toy made with this yarn is about 18 cm .  \r\n 

For training you'll need a dataset of `(input, label)` pairs. Where `input` and 
`label` are sequences. At each time step the input is the current character and the label is the next character. 

Here's a function that takes a sequence as input, duplicates, and shifts it to align the input and label for each timestep:

In [23]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [24]:
dataset = sequences.map(split_input_target)

In [25]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'monkey materials and tools \r\n 1 .  yarn :  yarnart jeans ,  colours :  black  ( 53 )  ,  light grey  ( 46 )  ,  dark grey  ( 28 )  ,  orange  ( 85 )  ,  white  ( 03 )  ,  yellow  ( 88 )  .  small quantity of white thin fluffy yarn  ( mohair ,  angora )  .  size of the toy made with this yarn is about 18 cm .  \r\n'
Target: b'materials and tools \r\n 1 .  yarn :  yarnart jeans ,  colours :  black  ( 53 )  ,  light grey  ( 46 )  ,  dark grey  ( 28 )  ,  orange  ( 85 )  ,  white  ( 03 )  ,  yellow  ( 88 )  .  small quantity of white thin fluffy yarn  ( mohair ,  angora )  .  size of the toy made with this yarn is about 18 cm .  \r\n 2'


### Create training batches

You used `tf.data` to split the text into manageable sequences. But before feeding this data into the model, you need to shuffle the data and pack it into batches.

In [26]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

## Build The Model

This section defines the model as a `keras.Model` subclass (For details see [Making new Layers and Models via subclassing](https://www.tensorflow.org/guide/keras/custom_layers_and_models)). 

This model has three layers:

* `tf.keras.layers.Embedding`: The input layer. A trainable lookup table that will map each character-ID to a vector with `embedding_dim` dimensions;
* `tf.keras.layers.GRU`: A type of RNN with size `units=rnn_units` (You can also use an LSTM layer here.)
* `tf.keras.layers.Dense`: The output layer, with `vocab_size` outputs. It outputs one logit for each character in the vocabulary. These are the log-likelihood of each character according to the model.

In [27]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_strings.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

print(vocab_size)

6393


In [28]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [29]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

For each character the model looks up the embedding, runs the GRU one timestep with the embedding as input, and applies the dense layer to generate logits predicting the log-likelihood of the next character:

![A drawing of the data passing through the model](https://github.com/tensorflow/text/blob/master/docs/tutorials/images/text_generation_training.png?raw=1)

Note: For training you could use a `keras.Sequential` model here. To  generate text later you'll need to manage the RNN's internal state. It's simpler to include the state input and output options upfront, than it is to rearrange the model architecture later. For more details see the [Keras RNN guide](https://www.tensorflow.org/guide/keras/rnn#rnn_state_reuse).

## Try the model

Now run the model to see that it behaves as expected.

First check the shape of the output:

In [30]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 6393) # (batch_size, sequence_length, vocab_size)


In the above example the sequence length of the input is `100` but the model can be run on inputs of any length:

In [31]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  1636608   
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  6552825   
                                                                 
Total params: 12,127,737
Trainable params: 12,127,737
Non-trainable params: 0
_________________________________________________________________


To get actual predictions from the model you need to sample from the output distribution, to get actual character indices. This distribution is defined by the logits over the character vocabulary.

Note: It is important to _sample_ from this distribution as taking the _argmax_ of the distribution can easily get the model stuck in a loop.

Try it for the first example in the batch:

In [32]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

This gives us, at each timestep, a prediction of the next character index:

In [33]:
sampled_indices

array([3954, 5012, 1508, 1721, 5922, 3863,  965, 1863, 2693, 4920, 2892,
       5993, 2602, 3344, 5755, 5417, 3792, 4217, 4093, 2513, 5063, 6041,
       4510, 1966, 1320, 3290, 2463, 5821,  213, 2916, 2166, 1864,  415,
       6088, 3639, 3465,  354, 2699, 1559, 1947, 4923, 4483, 2889, 4618,
       1251, 3173,   25, 6243,  431, 4609, 4381, 3584, 2847, 5697, 1052,
        490, 4390, 5987, 4542, 2007, 2937,  226, 2324, 2414,  599, 3205,
       6222,  269, 4100, 3126, 3952, 4568, 1530, 4549, 4290, 6002, 2382,
       3835, 1377, 2092, 1064, 5407, 5704, 2027, 5087,  403, 5431, 2802,
        399, 3747, 6085, 6216, 1863, 3065, 3164, 3711,  169, 4700, 4206,
       1673])

Decode these to see the text predicted by this untrained model:

In [34]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b' ( 30 )  \r\n rnd 6  :   ( 4 sc ,  inc )  *   ( 36 )  \r\n rnd 7  :   ( 5 sc ,  inc )  *   ( 42 )  \r\n rnd 8  :  through back loops only  ( 6 sc ,  inc )  *   ( 48 )  \r\n rnd 9  :   ( 5 sc ,  dec )  *   ( 42 )  \r\n rnd 10'

Next Char Predictions:
 b'much ruby begins burgundy toys millio 713 ch71 f1 rnd52 formaran twice engagement insides textile some materiales paddington numbers due sc14 until preferably closest apart; inc; domain throuh 140g free cows ch8 21hdc v26 lilo k507 1dec f5 black; claws rnd55 poopybutthole forma r10 alpaca her 003 wider 22rnd quadruple pig leg! fluffier tapes 83 28dc pincushion tutorial processed code73 fultterby 14rnd decreace dino 3417 hol whenever 165c obliquely hat; mrs pucca bermudas project paw; types detail metallic asembly continious 87 softi tassels color; sc54 20p spa firmly 20csc making v23 wet ch71 guideline help luck 12cm rd16 owner broom'


## Train the model

At this point the problem can be treated as a standard classification problem. Given the previous RNN state, and the input this time step, predict the class of the next character.

### Attach an optimizer, and a loss function

The standard `tf.keras.losses.sparse_categorical_crossentropy` loss function works in this case because it is applied across the last dimension of the predictions.

Because your model returns logits, you need to set the `from_logits` flag.


In [35]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [36]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 6393)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(8.762515, shape=(), dtype=float32)


A newly initialized model shouldn't be too sure of itself, the output logits should all have similar magnitudes. To confirm this you can check that the exponential of the mean loss is approximately equal to the vocabulary size. A much higher loss means the model is sure of its wrong answers, and is badly initialized:

In [37]:
tf.exp(example_batch_mean_loss).numpy()

6390.163

Configure the training procedure using the `tf.keras.Model.compile` method. Use `tf.keras.optimizers.Adam` with default arguments and the loss function.

In [38]:
model.compile(optimizer='adam', loss=loss)

### Configure checkpoints

Use a `tf.keras.callbacks.ModelCheckpoint` to ensure that checkpoints are saved during training:

In [39]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

### Execute the training

To keep training time reasonable, use 10 epochs to train the model. In Colab, set the runtime to GPU for faster training.

In [40]:
EPOCHS = 30

In [41]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/30
371/371 [==============================] - 62s 154ms/step - loss: 3.3447
Epoch 2/30
371/371 [==============================] - 52s 132ms/step - loss: 2.2098
Epoch 3/30
371/371 [==============================] - 50s 130ms/step - loss: 1.8907
Epoch 4/30
371/371 [==============================] - 49s 126ms/step - loss: 1.6608
Epoch 5/30
371/371 [==============================] - 48s 126ms/step - loss: 1.4729
Epoch 6/30
371/371 [==============================] - 48s 125ms/step - loss: 1.3043
Epoch 7/30
371/371 [==============================] - 49s 128ms/step - loss: 1.1543
Epoch 8/30
371/371 [==============================] - 50s 131ms/step - loss: 1.0215
Epoch 9/30
371/371 [==============================] - 49s 127ms/step - loss: 0.9060
Epoch 10/30
371/371 [==============================] - 50s 129ms/step - loss: 0.8064
Epoch 11/30
371/371 [==============================] - 51s 132ms/step - loss: 0.7218
Epoch 12/30
371/371 [==============================] - 52s 132ms/step - lo

## Generate text

The simplest way to generate text with this model is to run it in a loop, and keep track of the model's internal state as you execute it.

![To generate text the model's output is fed back to the input](https://github.com/tensorflow/text/blob/master/docs/tutorials/images/text_generation_sampling.png?raw=1)

Each time you call the model you pass in some text and an internal state. The model returns a prediction for the next character and its new state. Pass the prediction and state back in to continue generating text.


The following makes a single step prediction:

In [42]:
class OneStep(tf.keras.Model):
  def __init__(self, model, strings_from_ids, ids_from_strings, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.strings_from_ids = strings_from_ids
    self.ids_from_strings = ids_from_strings

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_strings = tf.strings.split(inputs, sep=" ")
    input_ids = self.ids_from_strings(input_strings) 
    # .to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[-1]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits 

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_strings = self.strings_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_strings, states

In [ ]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # # Create a mask to prevent "[UNK]" from being generated.
    # skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    # sparse_mask = tf.SparseTensor(
    #     # Put a -inf at each bad index.
    #     values=[-float('inf')]*len(skip_ids),
    #     indices=skip_ids,
    #     # Match the shape to the vocabulary
    #     dense_shape=[len(ids_from_chars.get_vocabulary())])
    # self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    temp = ""

    for i, each in enumerate(inputs):
      if each in "()[]{}.,*#'\"=-+":
        temp +=  f" {each} "
      else:
        temp += each
    text = temp
    text = re.sub('\\r\\n|\\r(?!\\n)', ' \r\n ', text)

    input_chars = tf.strings.split(text, sep=" ")
    input_ids = self.ids_from_chars(input_chars)

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits #+ self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [43]:
one_step_model = OneStep(model, strings_from_ids, ids_from_strings)

Run it in a loop to generate some text. Looking at the generated text, you'll see the model knows when to capitalize, make paragraphs and imitates a Shakespeare-like writing vocabulary. With the small number of training epochs, it has not yet learned to form coherent sentences.

In [80]:
start = time.time()
states = None
next_string = ['cat']
result = [next_string]

for n in range(1000):
  next_string, states = one_step_model.generate_one_step(next_string, states=states)
  result.append(next_string)

result = tf.strings.join(result, separator=" ")
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

cat cheeks  ( 2 parts )  
 white :  
 1 .  magic ring 6sc 
 2 - 8 .  6sc ,  1ch ,  w -  6sc ,  inc ,   ( 8sc ,  inc ) 2  ( 3sc ,  inc )  2 
 close ,  amigurumi character is another .  
 teddy ears .   ( brown pieces )  
 crochet hook :  
 1 .  6sc in mr 
 2 .  6inc  ( 12 )  
 3 .   ( 1sc ,  inc )   *  3  ( 15 )  
 4 .   ( 2sc ,  inc )   *  3 ,  5sc  ( 19 )  
 5 .  21sc 
 6 .   ( 2sc ,  inc )   *  5  ( 20 )  
 4 .   ( 2sc ,  inc )   *  8  ( 32 )  
 5 .   ( 3sc ,  inc )   *  3  ( 30 )  
 6 .   ( 4sc ,  inc )   *  6  ( 48 )  
 7 .   ( 7sc , inc )  * 6  ( 54 )  
 8 - 16 .  sc around  ( 60 )  
 17 .   ( 5sc ,  dec )  * 6  ( 48 )  
 18 .   ( 6sc , dec )  * 6  ( 48 )  
 19 .   ( 5sc ,  dec )   *  6  ( 48 )  
 20 .   ( 6sc ,  dec )   *  6  ( 42 )  
 15 .   ( 5sc ,  dec )   *  6  ( 36 )  
 19 .   ( 4sc ,  dec )   *  6  ( 30 )  
 20 .   ( 3sc ,  dec )   *  6  ( 24 )  
 21 .   ( sc ,  dec )  * 6  ( 18 )  
 22 .  sc in all the st  ( 18 )  
 fasten off ,  leaving a long tail for sewing .  
 bear ea

The easiest thing you can do to improve the results is to train it for longer (try `EPOCHS = 30`).

You can also experiment with a different start string, try adding another RNN layer to improve the model's accuracy, or adjust the temperature parameter to generate more or less random predictions.

If you want the model to generate text *faster* the easiest thing you can do is batch the text generation. In the example below the model generates 5 outputs in about the same time it took to generate 1 above. 

In [ ]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result, '\n\n' + '_'*80)
print('\nRun time:', end - start)

In [ ]:
import keras

one_step_model.save_weights("weights.h5")
reconstructed_model = keras.models.load_model("weights.h5")


ValueError: ignored

## Export the generator

This single-step model can easily be [saved and restored](https://www.tensorflow.org/guide/saved_model), allowing you to use it anywhere a `tf.saved_model` is accepted.

In [46]:
tf.saved_model.save(one_step_model, 'one_step_8')
one_step_reloaded = tf.saved_model.load('one_step_8')

In [ ]:
states = None
next_string = ['bat']            
result = [next_string]

for n in range(1000):
    next_string, states = one_step_reloaded.generate_one_step(next_string, states=states)
    result.append(next_string)

result = tf.strings.join(result, separator=" ")
end = time.time()
print((result[0].numpy().decode('utf-8'), '\n\n' + '_'*80))

ValueError: ignored

In [ ]:
import os

from google.colab import files

saveModelPath = "/content/one_step_8"
os.system("zip -r "+saveModelPath  + ".zip " +saveModelPath)
files.download(saveModelPath + ".zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
states = None
next_char = ['ROMEO:']
result = [next_char]

for n in range(1000):
  next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
  result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))

ValueError: ignored

## Advanced: Customized Training

The above training procedure is simple, but does not give you much control.
It uses teacher-forcing which prevents bad predictions from being fed back to the model, so the model never learns to recover from mistakes.

So now that you've seen how to run the model manually next you'll implement the training loop. This gives a starting point if, for example, you want to implement _curriculum  learning_ to help stabilize the model's open-loop output.

The most important part of a custom training loop is the train step function.

Use `tf.GradientTape` to track the gradients. You can learn more about this approach by reading the [eager execution guide](https://www.tensorflow.org/guide/eager).

The basic procedure is:

1. Execute the model and calculate the loss under a `tf.GradientTape`.
2. Calculate the updates and apply them to the model using the optimizer.

In [ ]:
class CustomTraining(MyModel):
  @tf.function
  def train_step(self, inputs):
      inputs, labels = inputs
      with tf.GradientTape() as tape:
          predictions = self(inputs, training=True)
          loss = self.loss(labels, predictions)
      grads = tape.gradient(loss, model.trainable_variables)
      self.optimizer.apply_gradients(zip(grads, model.trainable_variables))

      return {'loss': loss}

The above implementation of the `train_step` method follows [Keras' `train_step` conventions](https://www.tensorflow.org/guide/keras/customizing_what_happens_in_fit). This is optional, but it allows you to change the behavior of the train step and still use keras' `Model.compile` and `Model.fit` methods.

In [ ]:
model = CustomTraining(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [ ]:
model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

In [ ]:
model.fit(dataset, epochs=1)

Or if you need more control, you can write your own complete custom training loop:

In [ ]:
EPOCHS = 10

mean = tf.metrics.Mean()

for epoch in range(EPOCHS):
    start = time.time()

    mean.reset_states()
    for (batch_n, (inp, target)) in enumerate(dataset):
        logs = model.train_step([inp, target])
        mean.update_state(logs['loss'])

        if batch_n % 50 == 0:
            template = f"Epoch {epoch+1} Batch {batch_n} Loss {logs['loss']:.4f}"
            print(template)

    # saving (checkpoint) the model every 5 epochs
    if (epoch + 1) % 5 == 0:
        model.save_weights(checkpoint_prefix.format(epoch=epoch))

    print()
    print(f'Epoch {epoch+1} Loss: {mean.result().numpy():.4f}')
    print(f'Time taken for 1 epoch {time.time() - start:.2f} sec')
    print("_"*80)

model.save_weights(checkpoint_prefix.format(epoch=epoch))